# Superstore Data Exploration


In [ ]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from typing import List

warnings.filterwarnings("ignore")

### Data Exploration Analysis

In [ ]:
# ---
# Function to read and validate data
# ---


def read_data(file_path: str) -> pd.DataFrame:
    """
    Reads a CSV file into a pandas DataFrame and performs initial data checks.
    Args:
            file_path (str): The path to the CSV file.
    Returns:
            pd.DataFrame: The loaded DataFrame after performing checks.
    Raises:
            FileNotFoundError: If the specified file does not exist.
            ValueError: If the DataFrame is empty.
    """
    try:
        open(file_path, "r").close()
    except FileNotFoundError:
        raise FileNotFoundError(f"The file at {file_path} was not found.")
    df = pd.read_csv(file_path)
    if df.empty:
        raise ValueError("The provided CSV file is empty.")
    print(f"Reading File from: {file_path}\n{'=' * 40}")
    print(f"Dataframe Info:\n{df.info()}")
    print(f"{'=' *50} \nUnique Values")
    print(df.nunique())
    print(f"{'=' *50} \nChecking Duplicated Values:")
    duplicated = df.duplicated().sum()
    if duplicated > 0:
        print(f"There are {duplicated} duplicated rows.")
        df.drop_duplicates(inplace=True)
        print(f"Rows remaining after drop: {len(df)}")
    else:
        print(f"No duplicates ✅\n{'=' * 40}")
    print(df.head())
    return df


df = read_data("../data/raw/Sample - Superstore.csv")

In [ ]:
# ---------------------------------------------------
# Check Missing Values
# ---------------------------------------------------


def check_missing_values(df: pd.DataFrame):
    print("Checking for Missing (Null/NaN) Values 🔎")
    null_counts = df.isnull().sum()
    null_percentage = (null_counts / len(df)) * 100

    missing_data_report = pd.DataFrame(
        {"Missing Count": null_counts, "Missing %": null_percentage}
    )

    # Filter to only show columns with missing values and sort
    missing_data_report = missing_data_report[
        missing_data_report["Missing Count"] > 0
    ].sort_values(by="Missing %", ascending=False)

    print("\nMissing Value Report:")

    if not missing_data_report.empty:
        print(
            f"Found {len(missing_data_report)} columns with missing values.🚨"
        )
        print("-" * 50)
        # Use .to_string() for clean, non-truncated printing
        print(missing_data_report.to_string(float_format="%.2f%%"))
        print("-" * 50)
    else:
        print("No missing values detected. ✅")


check_missing_values(df)

In [ ]:
# ---------------------------------------------------
# EDA - Step 4: Study Variables
# ---------------------------------------------------
def plot_numerical_distro(
    df: pd.DataFrame, num_cols: List[str], bins: int = 30
):
    for col in num_cols:
        print(f"\n --- Analysis for Column {col} ---")

        fig, axes = plt.subplots(1, 2, figsize=(14, 5))

        # 1. Histogram (for distribution shape)
        sns.histplot(
            df[col].dropna(), kde=True, bins=bins, ax=axes[0], color="skyblue"
        )
        axes[0].set_title(f"Histogram & KDE for {col}")
        axes[0].set_xlabel(col)

        # 2. Box Plot (for central tendency and outliers)
        sns.boxplot(y=df[col].dropna(), ax=axes[1], color="lightcoral")
        axes[1].set_title(f"Box Plot for {col}")
        axes[1].set_ylabel(col)

        plt.tight_layout()
        plt.show()


def plot_categorical_variable(
    df: pd.DataFrame, cat_cols: List[str], bins: int = 30
):
    for col in cat_cols:
        print(f"\n --- Analysis for Column {col} ---")
        plt.figure(figsize=(10, 6))
        sns.countplot(
            y=df[col].dropna(),
            order=df[col].value_counts().index,
            palette="viridis",
        )
        for container in plt.gca().containers:
            plt.bar_label(container)

        plt.title(f"Frequency Count of Categories for {col}", fontsize=14)
        plt.xlabel("Count", fontsize=12)
        plt.ylabel(col, fontsize=12)
        plt.grid(axis="x", alpha=0.5)
        plt.tight_layout()
        plt.show()


def variable_study(df: pd.DataFrame, status: str = "num"):
    if status == "num":
        print(f"{'='*50} \n Numerical Variable Study")
        print(df.describe().T)
        numerical_var = df.select_dtypes(include="number")
        plot_numerical_distro(df, numerical_var)
    if status == "cat":
        print(f"{'='*50} \n Categorical Variable Study")
        categorical_var = df.select_dtypes(exclude="number")
        plot_categorical_variable(df, categorical_var)
    else:
        print("No categorical or Numerical Variable")


variable_study(df, "num")

In [ ]:
variable_study(df, "cat")

### Formatting


In [ ]:
# ---------------------------------------------------
# Convert Date Columns to Datetime
# ---------------------------------------------------


def convert_to_datetime(df: pd.DataFrame, date_columns: list) -> pd.DataFrame:
    """
    Converts specified columns in the DataFrame to datetime format.
    Args:
        df (pd.DataFrame): The input DataFrame.
        date_columns (list): List of column names to convert to datetime.
    Returns:
        pd.DataFrame: The DataFrame with converted date columns.
    """
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors="coerce")
            if df[col].isnull().any():
                print(
                    f"Warning: Some values in column '{col}' could not be converted and are set to NaT."
                )
        else:
            print(f"Column '{col}' not found in DataFrame.")
    return df


df_2 = convert_to_datetime(df, ["Order Date", "Ship Date"])

In [ ]:
# ---------------------------------------------------
# Standardize Column Names
# ---------------------------------------------------


def column_names_standardize(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardizes column names by converting to lowercase and replacing spaces with underscores.
    Args:
        df (pd.DataFrame): The input DataFrame.
    Returns:
        pd.DataFrame: The DataFrame with standardized column names.
    """
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]
    return df


df_3 = column_names_standardize(df_2)

In [ ]:
# ---------------------------------------------------
# Data Validation
# ---------------------------------------------------


def validate_data(df: pd.DataFrame):
    """
    Validates data by checking for negative values in specific columns.
    Args:
        df (pd.DataFrame): The input DataFrame.
    """
    numeric_cols = ["sales", "profit", "quantity", "discount"]
    for col in numeric_cols:
        if col in df.columns:
            negative_count = (df[col] < 0).sum()
            if negative_count > 0:
                print(
                    f"Warning: Column '{col}' contains {negative_count} negative values."
                )
            else:
                print(f"Column '{col}' has no negative values. ✅")
        else:
            print(f"Column '{col}' not found in DataFrame.")
    if df["discount"].max() > 1:
        print("Warning: 'discount' column has values greater than 1 (100%).")
    elif df["discount"].min() < 0:
        print("Warning: 'discount' column has negative values.")
    else:
        print(
            "Column 'discount' column values are within the expected range (0 to 1). ✅"
        )
    print(
        "All order dates are bigger than ship dates. ✅"
        if (df["order_date"] <= df["ship_date"]).all()
        else "Warning: Some order dates are after ship dates.🚨"
    )


validate_data(df_3)

In [ ]:
df_3.to_csv("../data/processed/Superstore-processed.csv")